In [12]:
import pandas as pd
import plotly.express as px
from os import path

In [13]:
df = pd.DataFrame()
for i in range(1200):
    df_part = pd.read_csv('simulations_day_output_{}.csv'.format(i), sep=';', header=None)
    df = pd.concat([df, df_part])
    
#     filename = 'simulations_day_break_output_{}.csv'.format(i)
#     if path.exists(filename):
#         df_part = pd.read_csv(filename, sep=';', header=None)
#         df = pd.concat([df, df_part])

In [14]:
df.shape

(2422, 4)

In [15]:
day_plan_cnt = 1440
df.columns = ['simulation_key', 'missing_plan_cnt_first_delay', 'missing_plan_cnt_planned_time', 'whole_process_delay']
df['controller_name'] = df.simulation_key.str.split('-', expand=True)[0]
df['break_strategy'] = df.simulation_key.str.split('-', expand=True)[1]
df['experiment_name'] = df.controller_name + '-' + df.break_strategy
df['simulation_index'] = df.simulation_key.str.split('-', expand=True)[2].astype(int)
df['produced_in_planned_time'] = day_plan_cnt - df.missing_plan_cnt_planned_time
df['whole_process_delay'] = df.whole_process_delay.astype(str).str.replace(',', '.').astype(float)
df['process_delay_minutes'] = df.whole_process_delay / 60
df['process_delay_hours'] = df.process_delay_minutes / 60

df.head()

,simulation_key,missing_plan_cnt_first_delay,missing_plan_cnt_planned_time,whole_process_delay,controller_name,break_strategy,experiment_name,simulation_index,produced_in_planned_time,process_delay_minutes,process_delay_hours
0,naive-skip_break-0,0,0,0.0,naive,skip_break,naive-skip_break,0,1440,0.0,0.0
1,naive-reorganization-0,0,0,0.0,naive,reorganization,naive-reorganization,0,1440,0.0,0.0
2,async-skip_break-0,0,0,0.0,async,skip_break,async-skip_break,0,1440,0.0,0.0
3,async-reorganization-0,0,0,0.0,async,reorganization,async-reorganization,0,1440,0.0,0.0
0,naive-skip_break-1,0,0,0.0,naive,skip_break,naive-skip_break,1,1440,0.0,0.0


In [24]:
df = df[df.simulation_index.isin(df[df.break_strategy=='reorganization'].simulation_index)]

In [31]:
df.groupby('experiment_name')[['produced_in_planned_time', 'process_delay_minutes']].describe()

produced_in_planned_time                                  \
                                        count         mean        std     min   
experiment_name                                                                 
async-reorganization                     11.0  1440.000000   0.000000  1440.0   
async-skip_break                         11.0  1440.000000   0.000000  1440.0   
naive-reorganization                     11.0  1433.272727  11.908744  1399.0   
naive-skip_break                         11.0  1433.181818  11.864385  1399.0   

                                                     process_delay_minutes  \
                         25%     50%     75%     max                 count   
experiment_name                                                              
async-reorganization  1440.0  1440.0  1440.0  1440.0                  11.0   
async-skip_break      1440.0  1440.0  1440.0  1440.0                  11.0   
naive-reorganization  1434.0  1436.0  1440.0  1440.0                  11.0   
naive-skip_break      1434.0  1436.0  1440.0  1440.0                  11.0   

                                                                      \
                          mean        std  min  25%    50%       75%   
experiment_name                                                        
async-reorganization  0.000000   0.000000  0.0  0.0  0.000  0.000000   
async-skip_break      0.000000   0.000000  0.0  0.0  0.000  0.000000   
naive-reorganization  5.893061  10.824749  0.0  0.0  3.548  5.023167   
naive-skip_break      5.944424  10.837786  0.0  0.0  3.548  5.214417   

                                 
                            max  
experiment_name                  
async-reorganization   0.000000  
async-skip_break       0.000000  
naive-reorganization  37.288500  
naive-skip_break      37.360333

In [32]:
df_summaries = pd.DataFrame(index=df.experiment_name.unique(),
                            columns=['perc_complete_plans', 'perc_zero_delay_plans',
                                    'avg_missing_steps', 'avg_accumulated_delay_minutes',
                                    'missing_steps_90_percentile', 'missing_steps_95_percentile', 'missing_steps_99_percentile',
                                    'missing_steps_70_percentile', 'missing_steps_80_percentile'])

for group_name, df_group in df.groupby('experiment_name'):
    df_summaries.loc[group_name, 'perc_complete_plans'] = df_group[df_group.missing_plan_cnt_planned_time == 0].shape[0] / df_group.shape[0] * 100
    df_summaries.loc[group_name, 'perc_complete_plans'] = str(round(df_summaries.loc[group_name, 'perc_complete_plans'], 1)) + ' %'
    df_summaries.loc[group_name, 'perc_zero_delay_plans'] = df_group[(df_group.whole_process_delay == 0)].shape[0] / df_group.shape[0] * 100
    df_summaries.loc[group_name, 'perc_zero_delay_plans'] = str(round(df_summaries.loc[group_name, 'perc_zero_delay_plans'], 1)) + ' %'
    df_summaries.loc[group_name, 'avg_missing_steps'] = df_group.missing_plan_cnt_planned_time.mean()
    df_summaries.loc[group_name, 'avg_accumulated_delay_minutes'] = df_group.process_delay_minutes.mean()
    df_summaries.loc[group_name, 'missing_steps_90_percentile'] = df_group.missing_plan_cnt_planned_time.quantile(q=0.9)
    df_summaries.loc[group_name, 'missing_steps_95_percentile'] = df_group.missing_plan_cnt_planned_time.quantile(q=0.95)
    df_summaries.loc[group_name, 'missing_steps_99_percentile'] = df_group.missing_plan_cnt_planned_time.quantile(q=0.99)
    df_summaries.loc[group_name, 'missing_steps_70_percentile'] = df_group.missing_plan_cnt_planned_time.quantile(q=0.70)
    df_summaries.loc[group_name, 'missing_steps_80_percentile'] = df_group.missing_plan_cnt_planned_time.quantile(q=0.80)

df_summaries

,perc_complete_plans,perc_zero_delay_plans,avg_missing_steps,avg_accumulated_delay_minutes,missing_steps_90_percentile,missing_steps_95_percentile,missing_steps_99_percentile,missing_steps_70_percentile,missing_steps_80_percentile
naive-skip_break,36.4 %,36.4 %,6.81818,5.94442,11,26,38,6,6
naive-reorganization,36.4 %,36.4 %,6.72727,5.89306,11,26,38,6,6
async-skip_break,100.0 %,100.0 %,0,0,0,0,0,0,0
async-reorganization,100.0 %,100.0 %,0,0,0,0,0,0,0


In [26]:
px.box(df, y='missing_plan_cnt_planned_time', color='experiment_name', title='Počet chybějících aut z plánu')

In [27]:
px.box(df, y='process_delay_minutes', color='experiment_name', title='Akumulovaný čas prostojů v minutách')

Maximalni delay uspesne vyrobene cele vyrobni rady

In [28]:
df[(df.whole_process_delay == 0) & (df.missing_plan_cnt_planned_time != 0)]

,simulation_key,missing_plan_cnt_first_delay,missing_plan_cnt_planned_time,whole_process_delay,controller_name,break_strategy,experiment_name,simulation_index,produced_in_planned_time,process_delay_minutes,process_delay_hours


In [29]:
df[(df.missing_plan_cnt_planned_time == 0)].sort_values('whole_process_delay', ascending=False)

,simulation_key,missing_plan_cnt_first_delay,missing_plan_cnt_planned_time,whole_process_delay,controller_name,break_strategy,experiment_name,simulation_index,produced_in_planned_time,process_delay_minutes,process_delay_hours
0,naive-skip_break-0,0,0,0.0,naive,skip_break,naive-skip_break,0,1440,0.0,0.0
1,naive-reorganization-0,0,0,0.0,naive,reorganization,naive-reorganization,0,1440,0.0,0.0
2,async-skip_break-721,0,0,0.0,async,skip_break,async-skip_break,721,1440,0.0,0.0
1,naive-reorganization-721,0,0,0.0,naive,reorganization,naive-reorganization,721,1440,0.0,0.0
0,naive-skip_break-721,0,0,0.0,naive,skip_break,naive-skip_break,721,1440,0.0,0.0
3,async-reorganization-689,0,0,0.0,async,reorganization,async-reorganization,689,1440,0.0,0.0
2,async-skip_break-689,0,0,0.0,async,skip_break,async-skip_break,689,1440,0.0,0.0
1,naive-reorganization-689,0,0,0.0,naive,reorganization,naive-reorganization,689,1440,0.0,0.0
0,naive-skip_break-689,0,0,0.0,naive,skip_break,naive-skip_break,689,1440,0.0,0.0
3,async-reorganization-621,0,0,0.0,async,reorganization,async-reorganization,621,1440,0.0,0.0


In [30]:
px.box(df[(df.missing_plan_cnt_planned_time == 0)], y='whole_process_delay', title='Délka prostoje v sekundách pro splněné plány')